### **Consigna**

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

In [28]:
!pip install gensim

In [29]:
import os
import platform

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence


In [30]:
# Descargar la carpeta de dataset
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [31]:
# Posibles bandas
os.listdir("./songs_dataset/")

['patti-smith.txt',
 'nickelback.txt',
 'missy-elliott.txt',
 'bjork.txt',
 'bob-dylan.txt',
 'beatles.txt',
 'lorde.txt',
 'blink-182.txt',
 'paul-simon.txt',
 'jimi-hendrix.txt',
 'amy-winehouse.txt',
 'prince.txt',
 'dickinson.txt',
 'eminem.txt',
 'leonard-cohen.txt',
 'radiohead.txt',
 'britney-spears.txt',
 'dj-khaled.txt',
 'adele.txt',
 'drake.txt',
 'lil-wayne.txt',
 'kanye-west.txt',
 'bruno-mars.txt',
 'janisjoplin.txt',
 'lin-manuel-miranda.txt',
 'bieber.txt',
 'ludacris.txt',
 'joni-mitchell.txt',
 'nicki-minaj.txt',
 'dr-seuss.txt',
 'notorious-big.txt',
 'r-kelly.txt',
 'bob-marley.txt',
 'kanye.txt',
 'nirvana.txt',
 'dolly-parton.txt',
 'rihanna.txt',
 'disney.txt',
 'nursery_rhymes.txt',
 'bruce-springsteen.txt',
 'michael-jackson.txt',
 'cake.txt',
 'notorious_big.txt',
 'al-green.txt',
 'Lil_Wayne.txt',
 'lady-gaga.txt',
 'Kanye_West.txt',
 'alicia-keys.txt',
 'johnny-cash.txt']

In [32]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/michael-jackson.txt', sep='/n', header=None)
df.head()

/tmp/ipython-input-2845004978.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,[Spoken Intro:]
1,You ever want something
2,that you know you shouldn't have
3,"The more you know you shouldn't have it,"
4,The more you want it


In [33]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9484


In [34]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [35]:
# Demos un vistazo
sentence_tokens[:2]

[['spoken', 'intro'], ['you', 'ever', 'want', 'something']]

### **Entrenamiento**

In [36]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class Callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [37]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(
    min_count=5,        # frecuencia mínima de palabra para incluirla en el vocabulario
    window=2,           # cant de palabras antes y desp de la predicha
    vector_size=300,    # dimensionalidad de los vectores
    negative=20,        # cantidad de negative samples... 0 es no se usa
    workers=1,          # si tienen más cores pueden cambiar este valor
    sg=1                # modelo 0: CBOW  1: Skipgram
)

In [38]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [39]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9484


In [40]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 1000


In [41]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(
    sentence_tokens,
    total_examples=w2v_model.corpus_count,
    epochs=20,
    compute_loss=True,
    callbacks=[Callback()]
)

Loss after epoch 0: 331950.15625
Loss after epoch 1: 227539.71875
Loss after epoch 2: 227819.5625
Loss after epoch 3: 222297.375
Loss after epoch 4: 193222.8125
Loss after epoch 5: 183347.125
Loss after epoch 6: 176326.25
Loss after epoch 7: 169747.875
Loss after epoch 8: 166771.625
Loss after epoch 9: 160761.375
Loss after epoch 10: 146806.875
Loss after epoch 11: 141884.5
Loss after epoch 12: 139681.25
Loss after epoch 13: 138101.5
Loss after epoch 14: 137683.0
Loss after epoch 15: 135786.5
Loss after epoch 16: 136097.25
Loss after epoch 17: 134627.25
Loss after epoch 18: 135685.75
Loss after epoch 19: 134597.25


(626341, 993780)

### **Palabras más relacionadas**

In [42]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["billie"], topn=10)

[('jean', 0.9854705929756165),
 ('son', 0.8961629271507263),
 ('lover', 0.8686242699623108),
 ('kid', 0.8457128405570984),
 ('sleep', 0.8197437524795532),
 ('dom', 0.8066651821136475),
 ('yours', 0.8008102178573608),
 ('sheldon', 0.8005437850952148),
 ('s', 0.7958450317382812),
 ('empty', 0.7907074093818665)]

Para la palabra "billie" la más relacionada es "jean" lo cual es un indicio de que el modelo está captando bien las relaciones dado que Billie Jean es una canción muy famosa de Michael Jackson y esa secuencia de dos palabras se repite constantemente en la canción.

In [43]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["bad"], topn=10)

[('shameone', 0.8981436491012573),
 ('smooth', 0.8740324378013611),
 ('laughing', 0.8125943541526794),
 ('fast', 0.8012159466743469),
 ('telling', 0.7683085203170776),
 ('bothered', 0.7678208947181702),
 ('easy', 0.7508718967437744),
 ('late', 0.746696949005127),
 ('everyday', 0.7322711944580078),
 ('talking', 0.7243325114250183)]

También para "bad" se observa que el modelo capta bien las relaciones, por ejemplo, la palabra "shameone" en la canción Bad siempre acompaña a "bad" en el estribillo.

In [44]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["beat"], topn=10)

[('magazine', 0.792034387588501),
 ('buy', 0.7834197878837585),
 ('fake', 0.752918541431427),
 ("couldn't", 0.7497268915176392),
 ('push', 0.744240939617157),
 ('seemed', 0.7409060597419739),
 ('waste', 0.7388805747032166),
 ('seems', 0.7358677387237549),
 ('read', 0.7328842878341675),
 ('feed', 0.7303219437599182)]

Para el caso de "beat" si no se encuentran palabras de la canción "Beat It" pero puede ser porque esta esté presente en otras canciones.

In [45]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["thriller"], topn=10)

[('winds', 0.8212677836418152),
 ('till', 0.8171051144599915),
 ('killer', 0.8169721364974976),
 ('darkness', 0.7892062067985535),
 ('breaking', 0.7787442207336426),
 ('s', 0.7718460559844971),
 ('big', 0.7713231444358826),
 ('injustice', 0.7671158909797668),
 ('road', 0.7557457685470581),
 ('lovers', 0.7529216408729553)]

In [46]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["black"], topn=10)

[('white', 0.9396378993988037),
 ('wrong', 0.7972461581230164),
 ('fortune', 0.7770756483078003),
 ('worth', 0.7764859199523926),
 ("thinkin'", 0.7687566876411438),
 ('three', 0.7634273171424866),
 ('tender', 0.7586590051651001),
 ('hate', 0.7563269734382629),
 ('vegetable', 0.7454425096511841),
 ("livin'", 0.7354086637496948)]

### **Palabras menos relacionadas**

In [47]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["billie"], topn=10)

[('siedah', 0.08880365639925003),
 ('stevie', 0.06315400451421738),
 ('refrain', 0.03693493828177452),
 ('uuh', 0.02843785099685192),
 ('cheater', -0.015202286653220654),
 ('pitbull', -0.05214293673634529),
 ('it', -0.13868212699890137),
 ("don't", -0.1431846022605896),
 ('stop', -0.1578296422958374),
 ('scream', -0.16708211600780487)]

In [48]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["bad"], topn=10)

[('siedah', 0.12770289182662964),
 ('pitbull', 0.08121579140424728),
 ('refrain', 0.049132056534290314),
 ('stevie', -0.029392410069704056),
 ('cheater', -0.06629934906959534),
 ('change', -0.0717666819691658),
 ('uuh', -0.08303966373205185),
 ('blow', -0.10409306734800339),
 ('will', -0.11063224077224731),
 ('inside', -0.12235770374536514)]

In [49]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["beat"], topn=10)

[('siedah', 0.09622729569673538),
 ('refrain', 0.03831429034471512),
 ('pitbull', -0.00527867441996932),
 ('stevie', -0.013235347345471382),
 ('uuh', -0.01904524490237236),
 ('cheater', -0.059486161917448044),
 ('not', -0.10853369534015656),
 ('been', -0.15916754305362701),
 ('here', -0.16139407455921173),
 ('jean', -0.16279557347297668)]

In [50]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["thriller"], topn=10)

[('siedah', 0.05236116424202919),
 ('uuh', 0.04429216310381889),
 ('refrain', 0.0350535623729229),
 ('stevie', 0.02821505255997181),
 ('pitbull', -0.013330912217497826),
 ('cheater', -0.04890734702348709),
 ('happens', -0.12231556326150894),
 ('let', -0.12501895427703857),
 ('whatever', -0.1350388377904892),
 ("g'on", -0.14043830335140228)]

In [51]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["black"], topn=10)

[('siedah', 0.10188192874193192),
 ('refrain', 0.09691575169563293),
 ('stevie', -0.01947762630879879),
 ('uuh', -0.028117647394537926),
 ('pitbull', -0.03136733919382095),
 ('cheater', -0.09709453582763672),
 ('tonight', -0.0988728255033493),
 ('got', -0.16357094049453735),
 ('stay', -0.16817636787891388),
 ('baby', -0.16904719173908234)]

Utilizando las mismas palabras que para ver las menos similares es interesante notar que hay muchas que se repiten: siedah, refrain, uuh, pitbull, cheater. Eso da a entener que eran palabras que solo existieron en una o pocas canciones de Michael Jackson.

### **Gráficos**

In [52]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [53]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [54]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

El modelo Word2Vec entrenado con las letras de Michael Jackson capturó relaciones semánticas, como la fuerte conexión entre "billie" y "jean" ("Billie Jean") y entre "bad" y "shameone" ("Bad"). La palabra "beat" mostró relaciones menos claras con la canción "Beat It", posiblemente por su uso en otros contextos. Palabras como "siedah", "refrain", "uuh", "pitbull" y "cheater" tuvieron baja similitud con los términos analizados, indicando su infrequentemente en el corpus. Las visualizaciones en 2D/3D ayudan a entender estas relaciones espaciales entre palabras. En resumen, se demostró la utilidad de Gensim para analizar relaciones semánticas en texto.